In [53]:
import numpy as np
# 隐马尔科夫模型参数
states = ['晴天', '雨天']
observations = ['散步', '购物', '清理房间']
start_prob = [0.6, 0.4]
# 状态转移矩阵：A =
# [
#  [0.7, 0.3],  # P(晴天|晴天)=0.7, P(雨天|晴天)=0.3
#  [0.4, 0.6]   # P(晴天|雨天)=0.4, P(雨天|雨天)=0.6
# ]
trans_prob = np.array([
    [0.7, 0.3],
    [0.4, 0.6]
])
# 观测概率矩阵：B =
# [
#  [0.4, 0.4, 0.2],  # P(散步|晴天)=0.4, P(购物|晴天)=0.4, P(清理房间|晴天)=0.2
#  [0.1, 0.3, 0.6]   # P(散步|雨天)=0.1, P(购物|雨天)=0.3, P(清理房间|雨天)=0.6
# ]
emission_prob = np.array([
    [0.4, 0.4, 0.2],
    [0.1, 0.3, 0.6]
])

def viterbi(obs_seq):
    # 将观察序列转换成数字编码，方便计算
    obs_idx = [observations.index(obs) for obs in obs_seq]
    # print("转换序列",obs_idx)
    # 观测序列长度
    T = len(obs_idx)
    # 初始状态矩阵长度
    N = trans_prob.shape[0]
    # 归一化
    log_start_prob = np.log(start_prob)
    log_trans_prob = np.log(trans_prob)
    log_emission_prob = np.log(emission_prob)
    
    V = np.zeros((T, N))
    B = np.zeros((T, N), dtype=np.int64)

    # 初始状态
    V[0] = log_start_prob + log_emission_prob[:, obs_idx[0]]

    for t in range(1, T):
        for j in range(N):
            prob = V[t - 1] + log_trans_prob[:, j]
            # 获取概览最大的
            max_idx = np.argmax(prob)
            # 记录当前路径的概率
            V[t, j] = prob[max_idx] + log_emission_prob[j, obs_idx[t]]
            # 记录路径
            B[t, j] = max_idx
            print(V)

    # 获取最大概率当作路径
    path = [np.argmax(V[-1])]
    for t in range(T - 2, -1, -1):
        path.append(B[t + 1, path[-1]])
    print(path)
    # 将数字编码转换成状态序列
    state_seq = [states[i] for i in path]
    return state_seq
# 观测序列 
obs_seq = ['散步', '散步', '购物', '清理房间']
state_seq = viterbi(obs_seq)
print('观察序列：', obs_seq)
print('最有可能的天气序列：', state_seq)


[[-1.42711636 -3.21887582]
 [-2.70008203  0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]]
[[-1.42711636 -3.21887582]
 [-2.70008203 -4.93367425]
 [ 0.          0.        ]
 [ 0.          0.        ]]
[[-1.42711636 -3.21887582]
 [-2.70008203 -4.93367425]
 [-3.97304771  0.        ]
 [ 0.          0.        ]]
[[-1.42711636 -3.21887582]
 [-2.70008203 -4.93367425]
 [-3.97304771 -5.10802764]
 [ 0.          0.        ]]
[[-1.42711636 -3.21887582]
 [-2.70008203 -4.93367425]
 [-3.97304771 -5.10802764]
 [-5.93916056  0.        ]]
[[-1.42711636 -3.21887582]
 [-2.70008203 -4.93367425]
 [-3.97304771 -5.10802764]
 [-5.93916056 -5.68784614]]
[1, 0, 0, 0]
观察序列： ['散步', '散步', '购物', '清理房间']
最有可能的天气序列： ['雨天', '晴天', '晴天', '晴天']


In [8]:
import xml.etree.ElementTree as ET
import jieba as jb

sentiment_dict = {}
with open('BosonNLP_sentiment_score.txt', 'r', encoding='utf-8') as f:
    for line in f:
        word, score = line.strip().split(" ")
        sentiment_dict[word] = float(score)

tree = ET.parse('ipad评论_情感分析.xml')
root = tree.getroot()
# # 等级词典
level_words = {}
flag = False
level_dict = {'极其|extreme / 最|most': 1, '很|very':0.8,'较|more':0.6,'稍|-ish':0.4,'欠|insufficiently':0.2,'超|over':0.9}
with open('程度级别词语（中文）.txt', 'r', encoding='gbk') as f:
    index = 0
    for line in f:
        line = line.strip()
        if "“" in line:
            line = line.split("\t")
            word = line[0]
            index = eval(line[1])
            leftIndex = word.find("“")
            rightIndex = word.find("”")
            word = word[leftIndex + 1: rightIndex]
            level = level_dict[word]
        elif index > 0:
            index -= 1
            level_words[line] = level
level_words["不"] = -1
# 计算情感值
for review in root:
    text = review.find('sentence').text
    score = 0
    count = 0
    list = jb.cut(text)
    
    for word in list:
        # print(word)
        if word in sentiment_dict:
            value = 1
            if word in level_words.keys():
                value = level_words[word]
            temp =  value * sentiment_dict[word]
            value = 1
            score += temp
            count += 1
    if count != 0:
        score /= count
    print('评论内容：', text)
    print('情感评分：', score) 


评论内容： #iPad3#windows 8  出厂了  但是微软公司 不让零售 额。。。。
情感评分： 0.16439247104487692
评论内容： #iPad3#神马配置都是渣，求一款平板，可以外接键盘，处理图片日常文件，可以玩主流PC游戏~。
情感评分： 0.015928147427286272
评论内容： #iPad3#话说，还有2个小时Ipad3就要上市了哦......
情感评分： 0.30853511160866426
评论内容： #iPad3#the new iPad无可厚非，技术是无价的，国货真不感恭维，像小米m1硬件很犀利，秒杀i9100;而价钱是三星的一半，如果小米卖4000，会有人买吗？
情感评分： 0.30002451135701436
评论内容： #iPad3#真的会有吗？
情感评分： -0.16898763811337503
评论内容： #ipad3#今天会随IPAD3一起发布IOS的系统更新吗？
情感评分： 0.3295539469962643
评论内容： #iPad3#从发布会邀请函上的图片来看真的有点诡异，是这个人手指太小了吗？
情感评分： 0.062046749410333224
评论内容： #iPad3#有USB接口和宽带接口吗？
情感评分： 0.21349390948827265
评论内容： #iPad3#很好很贵很xx，香港人用美元吗？？
情感评分： 0.10810675526193997
评论内容： #iPad3#iPad2与iPad3价格有区别吗？
情感评分： 0.3343584682060909
评论内容： #ipad3#苹果宣布3月7日发布iPad3苹果公司正式向媒体发布邀请函，将于太平洋时间3月7日上午10点在旧金山芳草地艺术中心（Yerba Buena Center for the Arts）召开下一代iPad的新闻发布会。[
情感评分： 0.7466413909475879
评论内容： 新一代iPad一定会在诸多方面有所升级，哪一个特性会触碰到你的小神经呢？
情感评分： -0.026159645007266683
评论内容： 中美物流涨价源于苹果大批#iPad3#秘送至美国http://url.cn/028Xn6  上周有报道称苹果已经开始从中国将iPad 3运送到美国。
情感评分： 0.76265141

In [9]:
import pandas as pd
from sklearn.metrics import r2_score

sms_df = pd.read_csv('../smsspamcollection/SMSSpamCollection', delimiter='\t', 
                     header=None, names=['class', 'text'])
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# 预处理数据
sms_df['text'] = sms_df['text'].apply(lambda x: x.lower().split())
vectorizer = CountVectorizer()
# 使用scikit-learn库提供的CountVectorizer类将单词列表转换为词频矩阵。
X = vectorizer.fit_transform(sms_df['text'].apply(lambda x: ' '.join(x)))
# 区分垃圾邮件与正常邮件
y = sms_df['class'].apply(lambda x: 1 if x == 'spam' else 0)
# print(X)
# 划分为训练集和测试集 

from sklearn.naive_bayes import MultinomialNB
def result(X_train,y_train,X_test,y_test):
# 训练朴素贝叶斯分类器
    clf = MultinomialNB()
    clf.fit(X_train, y_train)

    # 在测试集上评估分类器
    accuracy = clf.score(X_test, y_test)
    y_pre=clf.predict(X_test)
    r2 = r2_score(y_pre, y_test)
    print('Accuracy:', accuracy)
    print("r_score",r2)
print("----------------使用5000+574-----------------")
X_train, X_test, y_train, y_test = X[:5000], X[5000:], y[:5000], y[5000:]
result(X_train,y_train,X_test,y_test)
print("----------------使用随机划分(8:2)-----------------")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
result(X_train,y_train,X_test,y_test)
print("---------------使用3000+2574----------------")
X_train, X_test, y_train, y_test = X[:3000], X[3000:], y[:3000], y[3000:]
result(X_train,y_train,X_test,y_test)


----------------使用5000+574-----------------
Accuracy: 0.993006993006993
r_score 0.9393039049235993
----------------使用随机划分(8:2)-----------------
Accuracy: 0.9781048097631012
r_score 0.8103207156529553
---------------使用3000+2574----------------
Accuracy: 0.9797822706065319
r_score 0.8246348307240448
